In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle

WORKING_DIRECTORY = '/media/dean/datastore/datasets/darknet_evaluate'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data', 'coco')
BDD10K_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'bdd10k_instances_val2014.json')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images','train2014')
WEIGHTS_DIRECTORY = os.path.join(WORKING_DIRECTORY,'trained_weights')
LABELS_DIRECTORY = os.path.join(COCO_DIRECTORY, 'labels','train2014')
CATEGORY_NAMES = os.path.join(WORKING_DIRECTORY, 'data', 'coco.bdd100k.names')

In [2]:
bdd10k_coco = COCO(BDD10K_COCO_ANNOTATIONS_FILE)
ann_ids = bdd10k_coco.getAnnIds()
print(len(ann_ids))

loading annotations into memory...
Done (t=1.33s)
creating index...
index created!
185578


In [ ]:
# For Run in Training Runs    
training_runs = os.listdir(os.path.join(WEIGHTS_DIRECTORY,'training_runs'))
all_runs = []
for run in training_runs:
    print('Evaluating mAP on Training Run:', os.path.join(WEIGHTS_DIRECTORY,'training_runs',run))
    # Grab iterations and sort checkpoints
    d = {}
    weights_files = glob.glob(os.path.join(WEIGHTS_DIRECTORY,'training_runs',run, '*.weights'))
    
    for fpath in weights_files:
        fname = os.path.split(fpath)[-1]
        iterations = fname[15:].split('.weights')[0]
        d[fname] = int(iterations)
    sorted_weights = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]))
    bdd_weights = OrderedDict()

    for weights_file in sorted_weights.keys():
        d = {}
        d['iterations'] = sorted_weights[weights_file]
        d['map_results_file'] = os.path.join(WEIGHTS_DIRECTORY, 'training_runs',run, weights_file+'.txt')
        map_results_file = d['map_results_file']

        if not os.path.exists(d['map_results_file']+'.backup'):
            # Create annotations if labels directory not found
            if not os.path.exists(os.path.join(COCO_DIRECTORY,'labels/train2014/manifast.txt')):
                #!java -jar $WORKING_DIRECTORY/data/cocotoyolo.jar "{BDD10K_COCO_ANNOTATIONS_FILE}" "{IMAGES_DIRECTORY}/" "all" "{COCO_DIRECTORY}/labels/train2014"
                yolo_convert_output = os.path.join(COCO_DIRECTORY, 'labels','convert2yolo_results.txt')
                !python3 $WORKING_DIRECTORY/convert2Yolo/example.py --datasets COCO --img_path "{IMAGES_DIRECTORY}" --label "{BDD10K_COCO_ANNOTATIONS_FILE}" --convert_output_path "{LABELS_DIRECTORY}" --img_type ".jpg" --manipast_path $LABELS_DIRECTORY --cls_list_file $CATEGORY_NAMES | tee -a $yolo_convert_output
                
            CURRENT_WEIGHT = os.path.join(WEIGHTS_DIRECTORY, 'training_runs',run, weights_file)
            outfile = !cd $WORKING_DIRECTORY && ./darknet detector map cfg/bdd100k.data cfg/yolov3-bdd100k.cfg $CURRENT_WEIGHT | tee -a $map_results_file
            with open(d['map_results_file']+'.backup',"w+") as f:
                outfile = '\n'.join(outfile)
                f.write(outfile)

        #Get mAP results from file
        class_stats = []
        map_stats = {}
        with open(d['map_results_file']+'.backup') as openfile:
            for line in openfile:
                for part in line.split():
                    if 'class_id' in part:
                        class_map = {}
                        id_tokens = line.split('class_id =')[1].split(',')
                        class_map['class_id'] = int(id_tokens[0].strip())
                        
                        if 'name =' in line:
                            name_tokens = line.split('name =')[1].split(',')
                            class_map['class_name'] = name_tokens[0]

                        if 'ap =' in line:
                            ap_tokens = line.split('ap =')[1].split(',')
                            class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                        class_stats.append(class_map)
                    elif '(mAP)' in part:
                        tokens = line.split('(mAP) =')[1]
                        map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
                    elif 'Detection' in part:
                        tokens = line.split(':')[1]
                        total_detection_time = str(''.join(tokens)).strip()
                        map_stats['total_detection_time'] = total_detection_time
        map_stats['class_stats'] = class_stats
        print(map_stats,'\n\n') 
        bdd_weights[weights_file] = map_stats

    # Make sure Dictionary is made properly
    all_runs.append((run, bdd_weights))
print(all_runs)
pickle_file = os.path.join(COCO_DIRECTORY, 'mAP_all_runs.pickle')
pickle_dict = {'all_runs': all_runs}
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)   

Evaluating mAP on Training Run: /media/dean/datastore/datasets/darknet_evaluate/trained_weights/training_runs/run2_oldanchors
{'total_detection_time': '296.000000 Seconds', 'mean_avg_precision': '3.02', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '9.18'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '0.02'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '13.60'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '1.34'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '1.82'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '0.00'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.02'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.01'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.09'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '4.13'}]} 


{'total_detection_time': '301.000000 Seconds', 'mean_avg_precision': '2.37', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '2.55'}, 

{'total_detection_time': '300.000000 Seconds', 'mean_avg_precision': '17.96', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '18.96'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '12.22'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '31.72'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '32.60'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '28.61'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '0.00'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '13.15'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '14.47'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '19.29'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '8.55'}]} 


{'total_detection_time': '304.000000 Seconds', 'mean_avg_precision': '16.37', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '15.18'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '16.41'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '27.

{'total_detection_time': '308.000000 Seconds', 'mean_avg_precision': '15.51', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '14.07'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '2.06'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '46.27'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '23.53'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '18.36'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '0.00'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '4.55'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.98'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '27.57'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '17.71'}]} 


{'total_detection_time': '313.000000 Seconds', 'mean_avg_precision': '16.70', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '18.51'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '2.97'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '45.49'

{'total_detection_time': '300.000000 Seconds', 'mean_avg_precision': '35.49', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '35.06'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '26.66'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '61.94'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '50.76'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '43.23'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '0.00'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '19.92'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '28.65'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '51.55'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '37.12'}]} 


{'total_detection_time': '265.000000 Seconds', 'mean_avg_precision': '36.02', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '34.30'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '27.70'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '62

In [ ]:
print(len(bdd_weights))

In [ ]:
import re

class_stats = []
map_stats = {}
with open("/media/dean/datastore/datasets/darknet_evaluate/trained_weights/yolov3-bdd100k_24864.weights.txt.backup") as openfile:
    #Get mAP results from file
    for line in openfile:
        for part in line.split():
            if 'class_id' in part:
                class_map = {}
                id_tokens = line.split('class_id =')[1].split(',')
                class_map['class_id'] = int(id_tokens[0].strip())

                name_tokens = line.split('name =')[1].split(',')
                class_map['class_name'] = name_tokens[0]

                if 'ap =' in line:
                    ap_tokens = line.split('ap =')[1].split(',')
                    class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                class_stats.append(class_map)
            elif '(mAP)' in part:
                tokens = line.split('(mAP) =')[1]
                map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
            elif 'Detection' in part:
                tokens = line.split(':')[1]
                total_detection_time = str(''.join(tokens)).strip()
                map_stats['total_detection_time'] = total_detection_time
    
    map_stats['class_stats'] = class_stats
print(map_stats,'\n\n')
#!cat /media/dean/datastore/datasets/darknet_evaluate/trained_weights/yolov3-bdd100k_24864.weights.txt.backup
    

In [ ]:
import os

class_stats = []
map_stats = {}
class_map = {}
with open("/Users/deanwebb/Desktop/yolov3-bdd100k_28416.weights.txt.backup", 'r') as openfile:
    line = next(openfile)
    while line:
        reg_match = RegExParser(line)

        if reg_match.class_id:
            class_map['class_id'] = reg_match.class_id.group(1)

        if reg_match.class_name:
            class_map['class_name'] = reg_match.class_name.group(1)

        if reg_match.class_ap:
            class_map['class_ap'] = reg_match.class_ap.group(1)

        if reg_match.mean_ap:
            map_stats['mean_avg_precision'] = reg_match.mean_ap.group(1)

        if reg_match.obj_thresh:
            map_stats['obj_thresh'] = reg_match.obj_thresh.group(1)

        if reg_match.precision:
            map_stats['precision'] = reg_match.precision.group(1)

        if reg_match.recall:
            map_stats['recall'] = reg_match.recall.group(1)

        if reg_match.f1:
            map_stats['f1_score'] = reg_match.f1.group(1)

        if reg_match.TP:
            map_stats['TP'] = reg_match.TP.group(1)

        if reg_match.FP:
            map_stats['FP'] = reg_match.FP.group(1)

        if reg_match.FN:
            map_stats['FN'] = reg_match.FN.group(1)

        if reg_match.avg_IoU:
            map_stats['avg_IoU'] = reg_match.avg_IoU.group(1)




    #Get mAP results from file
    for line in openfile:
        for part in line.split():
            if 'class_id' in part:
                class_map = {}
                id_tokens = line.split('class_id =')[1].split(',')
                class_map['class_id'] = int(id_tokens[0].strip())

                    name_tokens = line.split('name =')[1].split(',')
                class_map['class_name'] = name_tokens[0]

                if 'ap =' in line:
                    ap_tokens = line.split('ap =')[1].split(',')
                    class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                class_stats.append(class_map)
            elif '(mAP)' in part:
                tokens = line.split('(mAP) =')[1]
                map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
            elif 'Detection' in part:
                tokens = line.split(':')[1]
                total_detection_time = str(''.join(tokens)).strip()
                map_stats['total_detection_time'] = total_detection_time
    map_stats['class_stats'] = class_stats
print(map_stats,'\n\n')


with open(filepath, 'r') as file:
    line = next(file)
    while line:
        reg_match = _RegExLib(line)

        if reg_match.school:
            school = reg_match.school.group(1)

        if reg_match.grade:
            grade = reg_match.grade.group(1)
            grade = int(grade)

        if reg_match.name_score:
            value_type = reg_match.name_score.group(1)
            line = next(file, None)
            while line and line.strip():
                number, value = line.strip().split(',')
                value = value.strip()
                dict_of_data = {
                    'School': school,
                    'Grade': grade,
                    'Student number': number,
                    value_type: value
                }
                data.append(dict_of_data)
                line = next(file, None)

        line = next(file, None)